rm reviews with too many non alphabet characters

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import json
from datetime import datetime

import gensim
import nltk
import pyLDAvis

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"          # disable huggingface warning

In [2]:
import platform
import torch

if platform.system() == 'Linux' or platform.system() == 'Windows':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('mps')        # m-series machine

print(device)

cuda


In [3]:
%load_ext autoreload

In [4]:
import sys

sys.path.append('../')

In [5]:
# load the dataset
# TODO: load any external dataset

%autoreload 2
from dataset_loader import GENRES, load_dataset

genre = GENRES.INDIE
unique_list = ['app_id', 'review_text']

dataset_folder = Path(f'../../dataset/topic_modelling/top_11_genres_unique_[{",".join(unique_list)}]')
dataset, dataset_path = load_dataset(genre, dataset_folder)

dataset.info(verbose=True)

Load dataset from: /root/FYP/NLP/dev-workspace/dataset/topic_modelling/top_11_genres_unique_[app_id,review_text]/01_indie.pkl



<class 'pandas.core.frame.DataFrame'>
Index: 740228 entries, 25636 to 4179608
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   index         740228 non-null  int64 
 1   app_id        740228 non-null  int64 
 2   app_name      740228 non-null  object
 3   review_text   740228 non-null  object
 4   review_score  740228 non-null  int64 
 5   review_votes  740228 non-null  int64 
 6   genre_id      740228 non-null  object
 7   category_id   740228 non-null  object
dtypes: int64(4), object(4)
memory usage: 50.8+ MB


In [6]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
# for each review, calculate the ratio between alphabets and non-alphabets

def calculate_nonalphabet_ratio(review: str) -> float:
    count = 0
    for char in review:
        if not char.isalpha():
            count += 1
    return count / (len(review) + 1e-5)

dataset['alphabet_ratio'] = dataset['review_text'].apply(calculate_nonalphabet_ratio)

dataset['alphabet_ratio'].describe([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

count    740228.000000
mean          0.228744
std           0.073642
min           0.000000
25%           0.201087
50%           0.218978
75%           0.240601
90%           0.276316
95%           0.313253
99%           0.459459
max           1.000000
Name: alphabet_ratio, dtype: float64

In [21]:
ttt = dataset[dataset['alphabet_ratio'] > 0.4]
ttt

,index,app_id,app_name,review_text,review_score,review_votes,genre_id,category_id,alphabet_ratio
29984,36928,102600,Orcs Must Die!,5 out of 5,1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.500000
29997,36941,102600,Orcs Must Die!,1,1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.999990
30200,37167,102600,Orcs Must Die!,'Get your [axe?] outta my face!' -Orc Archer ...,1,1,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.411765
30224,37191,102600,Orcs Must Die!,Nice game... !,1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.428571
30226,37194,102600,Orcs Must Die!,"Really fun, I love it :) 8/10",1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.448276
...,...,...,...,...,...,...,...,...,...
4177234,6413605,99700,NightSky,10/10,1,0,"[4, 23, 2]",[2],0.999998
4177236,6413607,99700,NightSky,🌟🌟🌟🌟🌟,1,0,"[4, 23, 2]",[2],0.999998
4177258,6413629,99700,NightSky,7/10 TJBs,1,0,"[4, 23, 2]",[2],0.555555
4177513,6413891,99700,NightSky,8 / 10,1,0,"[4, 23, 2]",[2],0.999998


In [22]:
ttt.loc[30200]['review_text']

"'Get your [axe?] outta my face!' -Orc Archer  10/10"

In [23]:
# data preprocessing

sys.path.append('../../sa')

%autoreload 2
import str_cleaning_functions


def cleaning(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))

def cleaning_strlist(str_list):
    str_list = list(map(lambda x: str_cleaning_functions.remove_links(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.remove_links2(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.clean(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.deEmojify(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.unify_whitespaces(x), str_list))
    return str_list

In [24]:
dataset_copy = dataset.copy()

cleaning(dataset_copy, 'review_text')

In [27]:
dataset_copy['alphabet_ratio'] = dataset_copy['review_text'].apply(calculate_nonalphabet_ratio)
dataset_copy['alphabet_ratio'].describe([0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

count    740228.000000
mean          0.223881
std           0.073168
min           0.000000
25%           0.199488
50%           0.215190
75%           0.234375
90%           0.265193
95%           0.298701
99%           0.444444
max           1.000000
Name: alphabet_ratio, dtype: float64

In [30]:
tt = dataset_copy[dataset_copy['alphabet_ratio'] > 0.44]
tt

,index,app_id,app_name,review_text,review_score,review_votes,genre_id,category_id,alphabet_ratio
29984,36928,102600,Orcs Must Die!,5 out of 5,1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.500000
29997,36941,102600,Orcs Must Die!,1,1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.999990
30201,37168,102600,Orcs Must Die!,,1,1,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.999990
30226,37194,102600,Orcs Must Die!,"Really fun, I love it :) 8/10",1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.448276
30235,37203,102600,Orcs Must Die!,★★★★,1,0,"[1, 23, 2]","[2, 22, 18, 23, 15, 25]",0.999998
...,...,...,...,...,...,...,...,...,...
4177092,6413460,99410,Garshasp: Temple of the Dragon,**,0,0,"[1, 23]","[2, 18]",0.999995
4177234,6413605,99700,NightSky,10/10,1,0,"[4, 23, 2]",[2],0.999998
4177258,6413629,99700,NightSky,7/10 TJBs,1,0,"[4, 23, 2]",[2],0.555555
4177513,6413891,99700,NightSky,8 / 10,1,0,"[4, 23, 2]",[2],0.999998


In [33]:
tt.iloc[234]['review_text']

'4/90000000000000000000000000 this game is good in my opinion'

In [25]:
X = dataset_copy['review_text'].values

In [ ]:
# remove empty strings

X = list(filter(lambda x: len(x) > 0, X))

In [ ]:
# check the length to be identical in the training script
print(len(X))
print(X[0])

740083
Take one part Faerie Solitaire and two parts Puzzle Quest and mix in a little Poker or Yahtzee for good measure and you will get something like Runespell: Overture. You're a changeling of some sort and you fight monsters and take quests in exchange for coin and buffs (which come in the form of power-up cards). There's a story but it's not the strongest element in the game. Like the Puzzle Quest games, your battles are determined by playing a mini-game. Instead of match-3 though, the game is a card game similar to poker in which making certain combinations of cards (pairs, 5 of a kind, full house, flush, straight) will do a certain amount of damage to your opponent, who is trying to do the same to you. The ability to steal some cards from your opponent, plus the limited number of moves you get per turn to move cards or play power-ups adds just enough strategy to the game to keep it interesting. Admittedly, the game can get a bit repetitive after a while and I found the dialogue o